In [1]:
from datetime import datetime

def compute_time_differences(input_string):
    lines = input_string.strip().split('\n')
    timestamps = []

    start_t = ''
    time_differences = []
    time_format = "%I:%M:%S %p"
    
    for line in lines:
        if line == '':
            continue
        if 'Clicked small' in line or 'Clicked long' in line:
            timestamp = line.split(' at ')[-1].split('(')[0]
            if 'AM' in line or 'PM' in line:
                start_t = datetime.strptime(timestamp, time_format)
            else:
                start_t = datetime.strptime(timestamp, '%H:%M:%S')
        if 'Clicked Mark Current' in line:
            timestamp = line.split(' at ')[-1].split('(')[0]
            if 'AM' in line or 'PM' in line:
                cur_t = datetime.strptime(timestamp, time_format)
            else:
                cur_t = datetime.strptime(timestamp, '%H:%M:%S')
            time_diff = (cur_t - start_t).total_seconds()
            time_differences.append(time_diff)

    return time_differences

In [ ]:

import subprocess
import os
import csv

test_len = 0
test_flag = False
for root, subdirs, files in os.walk('./'):
  for subdir in subdirs:
    if test_flag:
      break
    if subdir.startswith('user_'):
      subdir_p = os.path.join(root, subdir)
      for _root, _subdirs, _files in os.walk(subdir_p):
        for _subdir in _subdirs:
          if test_flag:
            break
          if _subdir.startswith('small') or _subdir.startswith('long'):
            js_dir = os.path.join(subdir_p, _subdir)
            js_file = os.path.join(subdir_p, _subdir, 'target_merged.js')
            # run `node js_file`; if there is no error, then the js file is valid
            # if there is an error, print this file path
            cmd = ['/home/ubuntu/.nvm/versions/node/v18.16.1/bin/node',js_file]
            process = subprocess.Popen(cmd, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd='./')
            stdout, stderr = process.communicate()
            process.wait()
            if stderr.decode().strip() != '':
              print(js_file)
              

In [ ]:
import os
import subprocess

def read_file(file_name):
  with open(file_name, 'r') as f:
    return f.read()

test_len = 0
test_flag = False
for root, subdirs, files in os.walk('./'):
  for subdir in subdirs:
    if test_flag:
      break
    if subdir.startswith('user_'):
      subdir_p = os.path.join(root, subdir)
      for _root, _subdirs, _files in os.walk(subdir_p):
        for _subdir in sorted(_subdirs):
          if test_flag:
            break
          if _subdir.startswith('small') or _subdir.startswith('long'):
            js_dir = os.path.join(subdir_p, _subdir)
            js_file = os.path.join(subdir_p, _subdir, 'target_merged.js')
            # run `node js_file`; if there is no error, then the js file is valid
            # if there is an error, print this file path
            cmd = ['/home/ubuntu/.nvm/versions/node/v18.16.1/bin/node',js_file]
            process = subprocess.Popen(cmd, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd='./')
            stdout, stderr = process.communicate()
            process.wait()
            if stderr.decode().strip() != '':
              # try to run the combined js file
              test_file = os.path.join(subdir_p, _subdir, 'target_test.js')
              userfix1_file = os.path.join(subdir_p, _subdir, 'target.err1.js.userfix.js')
              userfix2_file = os.path.join(subdir_p, _subdir, 'target.err2.js.userfix.js')
              userfix_file = None
              if os.path.exists(userfix1_file): userfix_file = userfix1_file
              elif os.path.exists(userfix2_file): userfix_file = userfix2_file
              if userfix_file is None:
                print(js_file, "no userfix file")
              else:
                test_code = read_file(test_file)
                userfix_code = read_file(userfix_file)
                assert "//TESTED_PROGRAM" in test_code
                combined_code = test_code.replace("//TESTED_PROGRAM", userfix_code)
                test_filepath = "/tmp/lalala.js"
                with open(test_filepath, 'w') as f:
                  f.write(combined_code)
                cmd = ['/home/ubuntu/.nvm/versions/node/v18.16.1/bin/node',test_filepath]
                process = subprocess.Popen(cmd, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd='./')
                stdout, stderr = process.communicate()
                process.wait()
                if stderr.decode().strip() != '':
                  print(js_file, "combined js file failed")
              

## Analyse the collected data

In [3]:
# analyse the time cost
def load_timecost_data(file):
  data = {}
  with open(file) as f:
    reader = csv.DictReader(f, fieldnames=['user_id', 'task1-spot', 'task1-total', 'task2-spot', 'task2-total',
                                            'task3-spot', 'task3-total', 'task1-fixed', 'task2-fixed', 'task3-fixed'])
    for idx, line in enumerate(reader):
      if idx == 0: continue
      if line['user_id'] == 'AVERAGE': break
      if line['user_id'].startswith('user_24'): continue
      data[line['user_id']] = {}
      data[line['user_id']]['task1-spot'] = int(line['task1-spot']) if line['task1-spot'] != '-' else 0
      data[line['user_id']]['task1-total'] = int(line['task1-total']) if line['task1-total'] != '-' else 0
      data[line['user_id']]['task2-spot'] = int(line['task2-spot']) if line['task2-spot'] != '-' else 0
      data[line['user_id']]['task2-total'] = int(line['task2-total']) if line['task2-total'] != '-' else 0
      data[line['user_id']]['task3-spot'] = int(line['task3-spot']) if line['task3-spot'] != '-' else 0
      data[line['user_id']]['task3-total'] = int(line['task3-total']) if line['task3-total'] != '-' else 0
      data[line['user_id']]['task1-fixed'] = int(line['task1-fixed']) if line['task1-fixed'] != '-' else 0
      data[line['user_id']]['task2-fixed'] = int(line['task2-fixed']) if line['task2-fixed'] != '-' else 0
      data[line['user_id']]['task3-fixed'] = int(line['task3-fixed']) if line['task3-fixed'] != '-' else 0
  print('Number of loaded data: ', len(data))
  return data
  
file_e = './_analysis/timing - exp-time.csv'
file_c = './_analysis/timing - control-time.csv'
data_e = load_timecost_data(file_e)
data_c = load_timecost_data(file_c)

Number of loaded data:  12
Number of loaded data:  12


In [13]:
# get the average and medium time cost for all task1 and tasks
def get_avg_med_timecost(data):
  spot_times_long, total_times_long = [], []
  spot_times_short, total_times_short = [], []
  for task in ['task1', 'task2', 'task3']:
    for user_id in data:
      if data[user_id][task+'-fixed'] == 0: continue
      if task == 'task1' or task == 'task2':
        spot_times_short.append(data[user_id][task+'-spot'])
        total_times_short.append(data[user_id][task+'-total'])
      else:
        spot_times_long.append(data[user_id][task+'-spot'])
        total_times_long.append(data[user_id][task+'-total'])
  print('Number of data to be averaged: short - ', len(spot_times_short), ', long - ', len(spot_times_long))
  tc_short_spot_avg, tc_short_total_avg = round(sum(spot_times_short)/len(spot_times_short)), round(sum(total_times_short)/len(total_times_short))
  tc_long_spot_avg, tc_long_total_avg = round(sum(spot_times_long)/len(spot_times_long)), round(sum(total_times_long)/len(total_times_long))
  print('Average time cost for short tasks: spot - ', tc_short_spot_avg, ', total - ', tc_short_total_avg, ', ratio - ', round(tc_short_spot_avg/tc_short_total_avg, 2))
  print('Average time cost for long tasks: spot - ', tc_long_spot_avg, ', total - ', tc_long_total_avg, ', ratio - ', round(tc_long_spot_avg/tc_long_total_avg, 2))
  tc_short_spot_med, tc_short_total_med = sorted(spot_times_short)[len(spot_times_short)//2], sorted(total_times_short)[len(total_times_short)//2]
  tc_long_spot_med, tc_long_total_med = sorted(spot_times_long)[len(spot_times_long)//2], sorted(total_times_long)[len(total_times_long)//2]
  print('Medium time cost for short tasks: spot - ', tc_short_spot_med, ', total - ', tc_short_total_med, ', ratio - ', round(tc_short_spot_med/tc_short_total_med, 2))
  print('Medium time cost for long tasks: spot - ', tc_long_spot_med, ', total - ', tc_long_total_med, ', ratio - ', round(tc_long_spot_med/tc_long_total_med, 2))
  
  # printing for copying to table
  print(tc_short_spot_avg, '\t', tc_short_total_avg, '\t', round(tc_short_spot_avg/tc_short_total_avg, 2))
  print(tc_long_spot_avg, '\t', tc_long_total_avg, '\t', round(tc_long_spot_avg/tc_long_total_avg, 2))
  print(tc_short_spot_med, '\t', tc_short_total_med, '\t', round(tc_short_spot_med/tc_short_total_med, 2))
  print(tc_long_spot_med, '\t', tc_long_total_med, '\t', round(tc_long_spot_med/tc_long_total_med, 2))
  
  # group data by the key of data; split the key using '_' and get the second part
  # group data to 8 groups by modulor 4
  new_group = [[] for _ in range(4)]
  for user_id in data:
    task_id = int(user_id.split('_')[1]) % 4
    new_group[task_id].append(data[user_id])
  
  for _g in new_group:
    # get the average and medium of spot and total time cost
    g_spot_times_long, g_total_times_long = [], []
    g_spot_time_short1, g_total_time_short1, g_spot_time_short2, g_total_time_short2 = [], [], [], []
    for user in _g:
      if user['task3-fixed'] != 0:
        g_spot_times_long.append(user['task3-spot'])
        g_total_times_long.append(user['task3-total'])
      if user['task1-fixed'] != 0:
        g_spot_time_short1.append(user['task1-spot'])
        g_total_time_short1.append(user['task1-total'])
      if user['task2-fixed'] != 0: 
        g_spot_time_short2.append(user['task2-spot'])
        g_total_time_short2.append(user['task2-total'])
    # print('Number of tasks: short1 - ', len(g_spot_time_short1), ', short2 - ', len(g_spot_time_short2), ', long - ', len(g_spot_times_long))
    # tc_short1_spot_avg, tc_short1_total_avg = round(sum(g_spot_time_short1)/len(g_spot_time_short1),2), round(sum(g_total_time_short1)/len(g_total_time_short1), 2)
    # tc_short2_spot_avg, tc_short2_total_avg = round(sum(g_spot_time_short2)/len(g_spot_time_short2),2), round(sum(g_total_time_short2)/len(g_total_time_short2), 2)
    # tc_longg_spot_avg, tc_longg_total_avg = round(sum(g_spot_times_long)/len(g_spot_times_long),2), round(sum(g_total_times_long)/len(g_total_times_long), 2)
    # print('Average time cost for short1 tasks: spot - ', tc_short1_spot_avg, ', total - ', tc_short1_total_avg, ', ratio - ', round(tc_short1_spot_avg/tc_short1_total_avg, 2))
    # print('Average time cost for short2 tasks: spot - ', tc_short2_spot_avg, ', total - ', tc_short2_total_avg, ', ratio - ', round(tc_short2_spot_avg/tc_short2_total_avg, 2))
    # print('Average time cost for long tasks: spot - ', tc_longg_spot_avg, ', total - ', tc_longg_total_avg, ', ratio - ', round(tc_longg_spot_avg/tc_longg_total_avg, 2))
    # tc_short1_spot_med, tc_short1_total_med = sorted(g_spot_time_short1)[len(g_spot_time_short1)//2], sorted(g_total_time_short1)[len(g_total_time_short1)//2]
    # tc_short2_spot_med, tc_short2_total_med = sorted(g_spot_time_short2)[len(g_spot_time_short2)//2], sorted(g_total_time_short2)[len(g_total_time_short2)//2]
    # tc_longg_spot_med, tc_longg_total_med = sorted(g_spot_times_long)[len(g_spot_times_long)//2], sorted(g_total_times_long)[len(g_total_times_long)//2]
    # print('Medium time cost for short1 tasks: spot - ', tc_short1_spot_med, ', total - ', tc_short1_total_med, ', ratio - ', round(tc_short1_spot_med/tc_short1_total_med, 2))
    # print('Medium time cost for short2 tasks: spot - ', tc_short2_spot_med, ', total - ', tc_short2_total_med, ', ratio - ', round(tc_short2_spot_med/tc_short2_total_med, 2))
    # print('Medium time cost for long tasks: spot - ', tc_longg_spot_med, ', total - ', tc_longg_total_med, ', ratio - ', round(tc_longg_spot_med/tc_longg_total_med, 2))
    
    # # printing for copying to table
    # print(tc_short1_spot_avg, '\t', tc_short1_total_avg, '\t', round(tc_short1_spot_avg/tc_short1_total_avg, 2))
    # print(tc_short2_spot_avg, '\t', tc_short2_total_avg, '\t', round(tc_short2_spot_avg/tc_short2_total_avg, 2))
    # print(tc_longg_spot_avg, '\t', tc_longg_total_avg, '\t', round(tc_longg_spot_avg/tc_longg_total_avg, 2))
    # print(tc_short1_spot_med, '\t', tc_short1_total_med, '\t', round(tc_short1_spot_med/tc_short1_total_med, 2))
    # print(tc_short2_spot_med, '\t', tc_short2_total_med, '\t', round(tc_short2_spot_med/tc_short2_total_med, 2)) 
    # print(tc_longg_spot_med, '\t', tc_longg_total_med, '\t', round(tc_longg_spot_med/tc_longg_total_med, 2))
  
  return tc_short_spot_avg, tc_short_total_avg, tc_long_spot_avg, tc_long_total_avg, tc_short_spot_med, tc_short_total_med, tc_long_spot_med, tc_long_total_med

print('Exp')
ge_tss_avg, ge_tst_avg, ge_tls_avg, ge_tlt_avg, ge_tss_med, ge_tst_med, ge_tls_med, ge_tlt_med = get_avg_med_timecost(data_e)
print('Control')
gc_tss_avg, gc_tst_avg, gc_tls_avg, gc_tlt_avg, gc_tss_med, gc_tst_med, gc_tls_med, gc_tlt_med = get_avg_med_timecost(data_c)

# print('Total decreasion percentage')
# per_short_total_med = round((gc_tst_med - ge_tst_med) / gc_tst_med, 2)
# per_long_total_med = round((gc_tlt_med - ge_tlt_med) / gc_tlt_med, 2)
# print('Decreasion percentage for median time cost: short - ', per_short_total_med, ', long - ', per_long_total_med)

print('Decreasion percentage')
per_tss_avg = round((gc_tss_avg - ge_tss_avg)/gc_tss_avg, 2)
per_tst_avg = round((gc_tst_avg - ge_tst_avg)/gc_tst_avg, 2)
per_tls_avg = round((gc_tls_avg - ge_tls_avg)/gc_tls_avg, 2)
per_tlt_avg = round((gc_tlt_avg - ge_tlt_avg)/gc_tlt_avg, 2)
per_tss_med = round((gc_tss_med - ge_tss_med)/gc_tss_med, 2)
per_tst_med = round((gc_tst_med - ge_tst_med)/gc_tst_med, 2)
per_tls_med = round((gc_tls_med - ge_tls_med)/gc_tls_med, 2)
per_tlt_med = round((gc_tlt_med - ge_tlt_med)/gc_tlt_med, 2)
print(f'Decreasion percentage for average time cost: short spot - {per_tss_avg}, short total - {per_tst_avg}, long spot - {per_tls_avg}, long total - {per_tlt_avg}')
print(f'Decreasion percentage for medium time cost: short spot - {per_tss_med}, short total - {per_tst_med}, long spot - {per_tls_med}, long total - {per_tlt_med}')

print('Pure fix time after spotting the error')
ge_sfix_avg = ge_tst_avg - ge_tss_avg
ge_lfix_avg = ge_tlt_avg - ge_tls_avg
ge_sfix_med = ge_tst_med - ge_tss_med
ge_lfix_med = ge_tlt_med - ge_tls_med
gc_sfix_avg = gc_tst_avg - gc_tss_avg
gc_lfix_avg = gc_tlt_avg - gc_tls_avg
gc_sfix_med = gc_tst_med - gc_tss_med
gc_lfix_med = gc_tlt_med - gc_tls_med
print('Fix decrease percentage')
per_sfix_avg = round((gc_sfix_avg - ge_sfix_avg)/gc_sfix_avg, 2)
per_lfix_avg = round((gc_lfix_avg - ge_lfix_avg)/gc_lfix_avg, 2)
per_sfix_med = round((gc_sfix_med - ge_sfix_med)/gc_sfix_med, 2)
per_lfix_med = round((gc_lfix_med - ge_lfix_med)/gc_lfix_med, 2)
print(f'Decreasion percentage for average fix time: short - {per_sfix_avg}, long - {per_lfix_avg}')
print(f'Decreasion percentage for medium fix time: short - {per_sfix_med}, long - {per_lfix_med}')

Exp
Number of data to be averaged: short -  22 , long -  12
Average time cost for short tasks: spot -  79 , total -  293 , ratio -  0.27
Average time cost for long tasks: spot -  258 , total -  465 , ratio -  0.55
Medium time cost for short tasks: spot -  60 , total -  235 , ratio -  0.26
Medium time cost for long tasks: spot -  286 , total -  365 , ratio -  0.78
79 	 293 	 0.27
258 	 465 	 0.55
60 	 235 	 0.26
286 	 365 	 0.78
Control
Number of data to be averaged: short -  21 , long -  8
Average time cost for short tasks: spot -  554 , total -  893 , ratio -  0.62
Average time cost for long tasks: spot -  1003 , total -  1520 , ratio -  0.66
Medium time cost for short tasks: spot -  452 , total -  739 , ratio -  0.61
Medium time cost for long tasks: spot -  1020 , total -  1255 , ratio -  0.81
554 	 893 	 0.62
1003 	 1520 	 0.66
452 	 739 	 0.61
1020 	 1255 	 0.81
Total decreasion percentage
Decreasion percentage for median time cost: short -  0.68 , long -  0.71
Decreasion percentag

In [12]:
# fill in the latex table (only the data for all short and long code; not for each group)
table_lex = r"""\begin{table}[ht]
\caption{\newadd{Mediums of spotting and total time cost (seconds) for all short and long programs with the number of quit tasks among two groups.}} 
\label{tab:userstudytimecost}
\begin{tabular}{|l|cc|cc|c|}
\hline
\multicolumn{1}{|c|}{\multirow{2}{*}{Group}} & \multicolumn{2}{c|}{Short (< 20 lines)}                                      & \multicolumn{2}{c|}{Long (> 150 lines)}                                       & \multirow{2}{*}{$N_{quit}$} \\ \cline{2-5}
\multicolumn{1}{|c|}{}                       & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} &                    \\ \hline
\textbf{$G_c$}                               & \multicolumn{1}{c|}{gc\_tss}             & gc\_tst              & \multicolumn{1}{c|}{gc\_tls}             & gc\_tlt              & 7              \\ \hline
\textbf{$G_e$}                               & \multicolumn{1}{c|}{ge\_tss}             & ge\_tst              & \multicolumn{1}{c|}{ge\_tls}             & ge\_tlt              & 2              \\ \hline
\end{tabular}
\end{table}"""

# print('Exp')
# ge_tss_avg, ge_tst_avg, ge_tls_avg, ge_tlt_avg, ge_tss_med, ge_tst_med, ge_tls_med, ge_tlt_med = get_avg_med_timecost(data_e)
# print('Control')
# gc_tss_avg, gc_tst_avg, gc_tls_avg, gc_tlt_avg, gc_tss_med, gc_tst_med, gc_tls_med, gc_tlt_med = get_avg_med_timecost(data_c)

table_lex_n = table_lex.replace('gc\_tss', str(gc_tss_med) + '\\textit{s}')\
                      .replace('gc\_tst', str(gc_tst_med) + '\\textit{s}')\
                        .replace('gc\_tls', str(gc_tls_med) + '\\textit{s}')\
                          .replace('gc\_tlt', str(gc_tlt_med) + '\\textit{s}')\
                            .replace('ge\_tss', str(ge_tss_med) + '\\textit{s}')\
                              .replace('ge\_tst', str(ge_tst_med) + '\\textit{s}')\
                                .replace('ge\_tls', str(ge_tls_med) + '\\textit{s}')\
                                  .replace('ge\_tlt', str(ge_tlt_med) + '\\textit{s}')
                                  
print(table_lex_n)

\begin{table}[ht]
\caption{\newadd{Mediums of spotting and total time cost (seconds) for all short and long programs with the number of quit tasks among two groups.}} 
\label{tab:userstudytimecost}
\begin{tabular}{|l|cc|cc|c|}
\hline
\multicolumn{1}{|c|}{\multirow{2}{*}{Group}} & \multicolumn{2}{c|}{Short (< 20 lines)}                                      & \multicolumn{2}{c|}{Long (> 150 lines)}                                       & \multirow{2}{*}{$N_{quit}$} \\ \cline{2-5}
\multicolumn{1}{|c|}{}                       & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} &                    \\ \hline
\textbf{$G_c$}                               & \multicolumn{1}{c|}{452\textit{s}}             & 739\textit{s}              & \multicolumn{1}{c|}{1020\textit{s}}             & 1255\textit{s}              & 7              \\ \hline
\textbf{$G_e$}                               & \multicolumn{1}{c|}{60\textit{s}

In [16]:
# average (appendix): fill in the latex table (only the data for all short and long code; not for each group)
table2_lex = r"""\begin{table}[ht]
\caption{\newadd{Average time for users to spot and fix mistakes in short and long programs and the number of quitted tasks.}} 
\label{tab:userstudytimecost}
\begin{tabular}{|l|cc|cc|c|}
\hline
\multicolumn{1}{|c|}{\multirow{2}{*}{\textbf{Group}}} & \multicolumn{2}{c|}{\textbf{Short Program}}                                      & \multicolumn{2}{c|}{\textbf{Long Program}}                                       & \multirow{2}{*}{$N_{unfin}$} \\ \cline{2-5}
\multicolumn{1}{|c|}{}                       & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} &                    \\ \hline
\textbf{$G_c$}                               & \multicolumn{1}{c|}{gc\_tss}             & gc\_tst              & \multicolumn{1}{c|}{gc\_tls}             & gc\_tlt              & 7              \\ \hline
\textbf{$G_e$}                               & \multicolumn{1}{c|}{ge\_tss}             & ge\_tst              & \multicolumn{1}{c|}{ge\_tls}             & ge\_tlt              & 2              \\ \hline
\end{tabular}
\end{table}"""

# print('Exp')
# ge_tss_avg, ge_tst_avg, ge_tls_avg, ge_tlt_avg, ge_tss_med, ge_tst_med, ge_tls_med, ge_tlt_med = get_avg_med_timecost(data_e)
# print('Control')
# gc_tss_avg, gc_tst_avg, gc_tls_avg, gc_tlt_avg, gc_tss_med, gc_tst_med, gc_tls_med, gc_tlt_med = get_avg_med_timecost(data_c)

table2_lex_n = table2_lex.replace('gc\_tss', str(gc_tss_avg) + '\\textit{s}')\
                      .replace('gc\_tst', str(gc_tst_avg) + '\\textit{s}')\
                        .replace('gc\_tls', str(gc_tls_avg) + '\\textit{s}')\
                          .replace('gc\_tlt', str(gc_tlt_avg) + '\\textit{s}')\
                            .replace('ge\_tss', str(ge_tss_avg) + '\\textit{s}')\
                              .replace('ge\_tst', str(ge_tst_avg) + '\\textit{s}')\
                                .replace('ge\_tls', str(ge_tls_avg) + '\\textit{s}')\
                                  .replace('ge\_tlt', str(ge_tlt_avg) + '\\textit{s}')
                                  
print(table2_lex_n)

\begin{table}[ht]
\caption{\newadd{Average time for users to spot and fix mistakes in short and long programs and the number of quitted tasks.}} 
\label{tab:userstudytimecost}
\begin{tabular}{|l|cc|cc|c|}
\hline
\multicolumn{1}{|c|}{\multirow{2}{*}{\textbf{Group}}} & \multicolumn{2}{c|}{\textbf{Short Program}}                                      & \multicolumn{2}{c|}{\textbf{Long Program}}                                       & \multirow{2}{*}{$N_{unfin}$} \\ \cline{2-5}
\multicolumn{1}{|c|}{}                       & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} & \multicolumn{1}{c|}{\textbf{$T_{spot}$}} & \textbf{$T_{total}$} &                    \\ \hline
\textbf{$G_c$}                               & \multicolumn{1}{c|}{554\textit{s}}             & 893\textit{s}              & \multicolumn{1}{c|}{1003\textit{s}}             & 1520\textit{s}              & 7              \\ \hline
\textbf{$G_e$}                               & \multicolumn{1}{c|}{79\textit{s}}    